## Damage Prediction in Materials with Machine Learning: Decision Tree, Random Forest, and Gradient Boosting Algorithms

First, import the necessary libraries.

In [57]:
# DataFrames and Arrays
import pandas as pd
import numpy as np

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_regression # Mutual Information regression for feature selection
from sklearn.model_selection import train_test_split # Function to split data into training and testing sets
from sklearn.model_selection import cross_val_score # Function for evaluating a model using cross-validation
from sklearn.model_selection import GridSearchCV # finding the optimal hyperparameters for a machine learning model

# Algorithms for regression tasks
from sklearn.tree import DecisionTreeRegressor, plot_tree # Decision Tree algorithm and tree plotting
from sklearn.ensemble import RandomForestRegressor # Random Forest algorithm
from xgboost import XGBRegressor # XGBoost algorithm 

Next, load the required 'CSV' files.

In [2]:
# Loading the required 'CSV' file
df_init = pd.read_csv('df.csv')
df_init.head(1)

,Nor_oper_time,Damage,Load_type,Material,Imp_J,Warp,Circ_lack_gl,Sq_lack_gl,Indent_kN,Lay_col
0,0.12,1.640000e-08,Interlam_sh,CFL,0,0,0,0,0,0


The DataFrame displayed above contains several important columns, including Operating Time, Damage, Load, Material, and Defect Types, among others. Notably, it also contains two columns with categorical values: `Load_type` and `Material`. These categorical variables need to be transformed into numerical format to enable their use in machine learning models, which typically require numerical input.

In [3]:
# Convert all categorical variables in the DataFrame to dummy/indicator variables
# This process creates binary columns for each category of every categorical variable
df_init = pd.get_dummies(df_init, drop_first=False).copy()
df_init.head(1)

,Nor_oper_time,Damage,Imp_J,Warp,Circ_lack_gl,Sq_lack_gl,Indent_kN,Lay_col,Load_type_Compr,Load_type_Interlam_sh,Load_type_Ios_sh,Load_type_Sh,Load_type_Tens,Material_CFL,Material_ST
0,0.12,1.640000e-08,0,0,0,0,0,0,0,1,0,0,0,1,0


Identify missing values.

In [4]:
# Check for missing values in the DataFrame
df_init.isnull().sum()

Nor_oper_time            0
Damage                   0
Imp_J                    0
Warp                     0
Circ_lack_gl             0
Sq_lack_gl               0
Indent_kN                0
Lay_col                  0
Load_type_Compr          0
Load_type_Interlam_sh    0
Load_type_Ios_sh         0
Load_type_Sh             0
Load_type_Tens           0
Material_CFL             0
Material_ST              0
dtype: int64

There are no missing values.

Next, let's split the dataset into features and labels.

In [5]:
# Separate the features and labels for model training
X = df_init.copy() # Features
y = X.pop("Damage") # Labels

Next, let's examine which features have the most significant impact on the target variable.

In [6]:
def mi_scor(X, y):
    """
    Calculate mutual information scores between each feature and the target variable,
    and return the scores as percentages of the total importance.
    
    Parameters:
    X (pd.DataFrame): Features dataset
    y (pd.Series): Target variable

    Returns:
    pd.DataFrame: DataFrame with features as index and their mutual information scores as percentages
    """
    mi_sc = mutual_info_regression(X, y) # Compute mutual information scores between each feature and the target variable
    mi_sc = pd.DataFrame(mi_sc, columns=["MI sc. import. in %"], index=X.columns) # Convert mutual inform. into a DataFrame
    mi_sc = mi_sc.sort_values(by="MI sc. import. in %", ascending=False) # Sort the DataFrame by mutual information
    
    return mi_sc/mi_sc.sum()*100 # Convert the mutual information scores to percentages of the total importance

mi_scor(X, y)

,MI sc. import. in %
Nor_oper_time,30.194943
Material_CFL,12.516065
Material_ST,12.419923
Load_type_Tens,11.231161
Indent_kN,6.225634
Load_type_Sh,5.472431
Imp_J,5.418548
Load_type_Interlam_sh,5.218290
Load_type_Compr,2.989878
Sq_lack_gl,2.703120


Notably, the variables `Nor_oper_time`, `Material_CFL`, `Material_ST` and `Load_type_Tens` have the greatest impact, collectively accounting for approximately 65% of the total importance.

Then, let's divide the data into training and testing sets.

In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Decision Tree

With our dataset prepared for training and testing, we can move forward with applying machine learning algorithms. The Decision Tree algorithm is one of the simplest and most intuitive models, utilizing a tree-like structure where each internal node represents a decision based on a feature. It partitions the data into subsets, leading to outcomes at the leaf nodes, ultimately aiming to make accurate predictions.

In [8]:
# Initialize the Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state=42)

# Define the parameter grid for GridSearchCV
param_grid_DT = {
    'max_depth': [None, 10, 11, 15, 16, 17, 18, 19, 20], # Depth of the tree
    'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # Minimum number of samples required to be at a leaf node
}

# Initialize GridSearchCV with the Decision Tree Regressor
# cv=5 specifies 5-fold cross-validation
# scoring='neg_mean_squared_error' means the minimization of the mean squared error
grid_search_DT = GridSearchCV(dt_regressor, param_grid_DT, cv=5, scoring='neg_mean_squared_error')
# Fit GridSearchCV to the training data
grid_search_DT.fit(X_train, y_train) 

# Display the best parameters and the best cross-validation mean squared error (MSE)
print("Best parameters:", grid_search_DT.best_params_)
print("Best cross-validation MSE: {:.5f}".format(-grid_search_DT.best_score_))

Best parameters: {'max_depth': 11, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best cross-validation MSE: 0.00147


The code above performs hyperparameter tuning for a Decision Tree Regressor to find the optimal combination of parameters that minimizes the mean squared error (MSE) through 5-fold cross-validation. With these best parameters, we can apply the Decision Tree model and evaluate its performance on the test data.

In [12]:
# Evaluate the final model on the test set
DT_final_model = grid_search_DT.best_estimator_ # Retrieve the best estimator
DT_final_model.fit(X_train, y_train)  # Fit the final model to the entire training set using the best parameters

# Evaluate the model's performance on the test set using the coefficient of determination (R² score)
test_CoD = DT_final_model.score(X_test, y_test)  
# Generate predictions on the test set
y_pr = DT_final_model.predict(X_test)

# Print the coefficient of determination and the mean squared error
print("Decision Tree: R² score on the test set: {:.3f}".format(test_CoD))
print("Decision Tree: MSE on the test set: {:.5f}".format(((y_test - y_pr)**2).mean()))

Decision Tree: R² score on the test set: 0.919
Decision Tree: MSE on the test set: 0.00166


### Random Forest

The next algorithm we'll explore is the Random Forest Regressor, which is an ensemble method known for its ability to improve prediction accuracy by combining multiple decision trees.

In [28]:
# Initialize the RandomForestRegressor
RF_model = RandomForestRegressor(random_state=42)

# This grid specifies the hyperparameters to be tuned for the RandomForestRegressor
param_grid_RF = {
    'n_estimators': [100, 150, 200, 250, 300], # Number of trees in the forest
    'max_features': [0.2, 0.4, 0.6, 0.8, 1], # Fraction of features to consider for each split
    'max_depth': [None, 5, 10, 15, 20], # Maximum depth of the trees
    'min_samples_split': [2, 5, 10, 15], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 3, 4] # Minimum number of samples required to be at a leaf node
}

# Setup GridSearchCV with the RandomForestRegressor and the parameter grid
grid_search_RF = GridSearchCV(estimator=RF_model, param_grid=param_grid_RF, cv=5, scoring='neg_mean_squared_error', 
                           verbose=2, n_jobs=-1)
# Fit GridSearchCV on the training data
grid_search_RF.fit(X_train, y_train)
    
# Retrieve the best parameters and the best cross-validation mean squared error
print("Best parameters:", grid_search_RF.best_params_)
print("Best cross-validation MSE: {:.5f}".format(-grid_search_RF.best_score_))

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits
Best parameters: {'max_depth': 10, 'max_features': 0.8, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 150}
Best cross-validation MSE: 0.00123


In [31]:
# Evaluate the final model on the test set
RF_final_model = grid_search_RF.best_estimator_ # Retrieve the best estimator
RF_final_model.fit(X_train, y_train)  # Fit the final model to the entire training set using the best parameters

# Evaluate the model's performance on the test set using the coefficient of determination (R² score)
test_CoD = RF_final_model.score(X_test, y_test)
# Generate predictions on the test set
y_pr = RF_final_model.predict(X_test)

# Print the coefficient of determination and the mean squared error
print("Decision Tree: R² score on the test set: {:.3f}".format(test_CoD))
print("Decision Tree: MSE on the test set: {:.5f}".format(((y_test - y_pr)**2).mean()))

Decision Tree: R² score on the test set: 0.935
Decision Tree: MSE on the test set: 0.00133


### Gradient Boosting

The last algorithm we'll explore is Gradient Boosting, which is an ensemble technique that builds models sequentially. Each new model corrects the errors of the previous ones by focusing on the residuals, or mistakes, made by the prior models.

In [36]:
# Define the parameter grid for XGBRegressor
param_grid_XGB = {
    'n_estimators': [10, 50, 100], # Number of boosting rounds to build
    'learning_rate': [0.15, 0.17, 0.19], # Step size shrinkage used to prevent overfitting
    'max_depth': [1, 3, 4, 5, 8], # Maximum depth of the individual trees
    'colsample_bytree': [0.90, 0.92, 0.94], # Fraction of features to be used for each tree
    'subsample': [0.96, 0.98, 1.0] # Fraction of samples used for fitting each tree
}

# Initialize the XGBRegressor
xgb = XGBRegressor(random_state=42)

# Setup GridSearchCV with the XGBRegressor and the parameter grid
grid_search_XGB = GridSearchCV(estimator=xgb, param_grid=param_grid_XGB, cv=5, scoring='neg_mean_squared_error', 
                               verbose=2, n_jobs=-1)

# Perform the grid search to find the best hyperparameters
grid_search_XGB.fit(X_train, y_train)

# Retrieve the best parameters and the best cross-validation mean squared error
print("Best parameters:", grid_search_XGB.best_params_)
print("Best cross-validation MSE: {:.5f}".format(-grid_search_XGB.best_score_))

Fitting 5 folds for each of 405 candidates, totalling 2025 fits
Best parameters: {'colsample_bytree': 0.94, 'learning_rate': 0.17, 'max_depth': 4, 'n_estimators': 50, 'subsample': 1.0}
Best cross-validation MSE: 0.00265


In [38]:
# Evaluate the final model on the test set
XGB_final_model = grid_search_XGB.best_estimator_ # Retrieve the best estimator
XGB_final_model.fit(X_train, y_train)  # Fit the final model to the entire training set using the best parameters

# Evaluate the model's performance on the test set using the coefficient of determination (R² score)
test_CoD = XGB_final_model.score(X_test, y_test)
# Generate predictions on the test set
y_pr = XGB_final_model.predict(X_test)

# Print the coefficient of determination and the mean squared error
print("Decision Tree: R² score on the test set: {:.3f}".format(test_CoD))
print("Decision Tree: MSE on the test set: {:.5f}".format(((y_test - y_pr)**2).mean()))

Decision Tree: R² score on the test set: 0.846
Decision Tree: MSE on the test set: 0.00315


Comparative analysis of algorithms shows that the Random Forest algorithm demonstrates the highest descriptive capability in modeling the patterns of damage accumulation and is preferred for use in this case.